In [46]:
from botocore import UNSIGNED
from botocore.config import Config
import boto3
import os
import pyart
from datetime import datetime, timedelta
import os
import glob
from datetime import datetime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import imageio

import matplotlib.cm as cm
import matplotlib.colors as mcolors

import osmnx as ox
from matplotlib.ticker import MultipleLocator

from math import cos, radians
import numpy as np
import numpy.ma as ma

In [8]:
# --- Configuration ---
station = 'KRTX'
start_time = datetime(2024, 8, 17, 21, 0)
end_time = datetime(2024, 8, 18, 3, 0)
output_dir = './radar_files'
desired_fields = ['reflectivity', 'velocity', 'cross_correlation_ratio']

# --- Setup ---
os.makedirs(output_dir, exist_ok=True)
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
bucket = 'noaa-nexrad-level2'

def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)

def download_radar_files(station, start_time, end_time):
    files = []
    start_date = start_time.date()
    end_date = end_time.date()

    for single_date in daterange(start_date, end_date):
        prefix = f"{single_date:%Y/%m/%d}/{station}/"
        print(f"Checking prefix: {prefix}")

        response = s3.list_objects_v2(Bucket=bucket, Prefix=prefix)
        if 'Contents' in response:
            for obj in response['Contents']:
                key = obj['Key']
                basename = os.path.basename(key)
                try:
                    # Parse time from filename, e.g. KRTX20230818_0104_V06
                    file_time = datetime.strptime(basename[4:17], "%Y%m%d_%H%M")
                    if start_time <= file_time <= end_time:
                        local_path = os.path.join(output_dir, basename)
                        if not os.path.exists(local_path):
                            print(f"Downloading {basename}")
                            s3.download_file(bucket, key, local_path)
                        files.append(local_path)
                except Exception as e:
                    # skip files with unexpected naming
                    continue
    return sorted(files)

# --- Run ---
files = download_radar_files(station, start_time, end_time)


Checking prefix: 2024/08/17/KRTX/
Checking prefix: 2024/08/18/KRTX/


In [50]:
def add_gridlines(ax, extent, spacing_lat=None, spacing_lon=None):
    lon_min, lon_max, lat_min, lat_max = extent
    if spacing_lat is None:
        spacing_lat = max((lat_max - lat_min) / 5, 0.01) 
    if spacing_lon is None:
        spacing_lon = max((lon_max - lon_min) / 5, 0.01)

    gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    import matplotlib.ticker as mticker
    import numpy as np
    gl.xlocator = mticker.FixedLocator(np.arange(lon_min, lon_max + spacing_lon, spacing_lon))
    gl.ylocator = mticker.FixedLocator(np.arange(lat_min, lat_max + spacing_lat, spacing_lat))

    gl.xlabel_style = {'size': 8}
    gl.ylabel_style = {'size': 8}

def domain_size_km(lat_min, lat_max, lon_min, lon_max):
    """Calculate approximate max dimension of domain in km."""
    avg_lat = (lat_min + lat_max) / 2
    lat_km = (lat_max - lat_min) * 111
    lon_km = (lon_max - lon_min) * 111 * cos(radians(avg_lat))
    return max(lat_km, lon_km)

def download_osm_features(lat_min, lat_max, lon_min, lon_max):
    size_km = domain_size_km(lat_min, lat_max, lon_min, lon_max)
    print('Size: ', size_km, 'km')
    
    # Thresholds (example, tweak to your liking)
    # Smaller size -> more detail
    road_detail_levels = [
        (5, ["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "residential", "service", "track", "path"]),
        (20, ["motorway", "trunk", "primary", "secondary", "tertiary", "unclassified", "residential"]),
        (50, ["motorway", "trunk", "primary", "secondary"]),
        (200, ["motorway", "trunk", "primary"]),
        (500, ["motorway", "trunk"]),
        (float('inf'), ["motorway"])
    ]

    city_detail_levels = [
        (50, ["city", "town", "village", "hamlet"]),
        (100, ["city", "town", "village"]),
        (150, ["city", "town"]),
        (500, ["city"])
    ]

    water_detail_levels = [
        (5, {"natural": ["water"], "waterway": ["river", "stream", "canal", "drain", "ditch"]}),
        (20, {"natural": ["water"], "waterway": ["river", "stream", "canal"]}),
        (500, {"natural": ["water"], "waterway": ["river"]}),
        (float('inf'), {"natural": ["water"]})
    ]

    def select_level(levels, size):
        for threshold, val in levels:
            if size <= threshold:
                return val
        return levels[-1][1]

    # Select tags based on domain size
    highway_tags = select_level(road_detail_levels, size_km)
    city_tags = select_level(city_detail_levels, size_km)
    water_tags = select_level(water_detail_levels, size_km)

    # Download roads
    print("Downloading roads with tags:", highway_tags)
    try:
        roads = ox.features.features_from_bbox(
            (lon_min, lat_min, lon_max, lat_max),
            tags={"highway": highway_tags}
        )
        roads = roads[roads.geometry.type.isin(["LineString", "MultiLineString"])]
    except Exception as e:
        print(f"⚠️ Failed to download roads: {e}")
        roads = None

    # Download water
    print("Downloading water features with tags:", water_tags)
    try:
        water = ox.features.features_from_bbox(
            (lon_min, lat_min, lon_max, lat_max),
            tags=water_tags
        )
        water = water[water.geometry.type.isin(["Polygon", "MultiPolygon", "LineString", "MultiLineString"])]
    except Exception as e:
        print(f"⚠️ Failed to download water features: {e}")
        water = None

    # Download cities
    print("Downloading cities with tags:", city_tags)
    try:
        cities = ox.features.features_from_bbox(
            (lon_min, lat_min, lon_max, lat_max),
            tags={"place": city_tags}
        )
        cities = cities[cities.geometry.type == "Point"]
    except Exception as e:
        print(f"⚠️ Failed to download cities: {e}")
        cities = None

    return roads, water, cities

    
def compute_osm_bbox(lat_min, lat_max, lon_min, lon_max, lat_rate, lon_rate, n_frames):
    # Compute full domain bounds after all shifts
    lat_max_final = lat_max + (n_frames - 1) * lat_rate
    lat_min_final = lat_min + (n_frames - 1) * lat_rate
    lon_max_final = lon_max + (n_frames - 1) * lon_rate
    lon_min_final = lon_min + (n_frames - 1) * lon_rate

    overall_lat_min = min(lat_min, lat_min_final)
    overall_lat_max = max(lat_max, lat_max_final)
    overall_lon_min = min(lon_min, lon_min_final)
    overall_lon_max = max(lon_max, lon_max_final)

    return overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max


def plot_field_sequence(
    data_dir: str,
    output_dir: str,
    field: str,
    start_time: datetime,
    end_time: datetime,
    lat_min: float = None,
    lat_max: float = None,
    lon_min: float = None,
    lon_max: float = None,
    lat_rate: float = 0.0,
    lon_rate: float = 0.0,
    dpi: int = 200,
    figsize: tuple = (6, 4),
    cmap: str = 'NWSRef',
    vmin: float = None,
    vmax: float = None,
    map_detail: str = '10m',
    features: bool = True,
    sweep: int = 0,
    bbox_inches: str = None
):
    frames_dir = os.path.join(output_dir, 'frames')
    os.makedirs(frames_dir, exist_ok=True)

    all_files = sorted(glob.glob(os.path.join(data_dir, '*')))
    def file_time(fn):
        s = os.path.basename(fn)
        return datetime.strptime(s[4:17], '%Y%m%d_%H%M')
    files = [f for f in all_files if (start_time <= file_time(f) <= end_time) and f.endswith('_V06')]
    files.sort(key=file_time)

    if not files:
        print(f"⚠️  No files found between {start_time} and {end_time} in {data_dir}")
        return

    if lat_min is None or lon_min is None:
        r0 = pyart.io.read(files[0])
        lats = r0.gate_latitude['data']
        lons = r0.gate_longitude['data']
        lat_min, lat_max = float(lats.min()), float(lats.max())
        lon_min, lon_max = float(lons.min()), float(lons.max())
        del r0

    overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max = compute_osm_bbox(
        lat_min, lat_max, lon_min, lon_max, lat_rate, lon_rate, len(files)
    )

    if features:
        road_gdf, water_gdf, city_gdf = download_osm_features(
            overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max
        )
    else:
        road_gdf, water_gdf, city_gdf = None, None, None

    domain_km = domain_size_km(overall_lat_min, overall_lat_max, overall_lon_min, overall_lon_max)

    frame_files = []
    for i, fn in enumerate(files):
        t = file_time(fn)
        print(t)
        c_lat_min = lat_min + i * lat_rate
        c_lat_max = lat_max + i * lat_rate
        c_lon_min = lon_min + i * lon_rate
        c_lon_max = lon_max + i * lon_rate
        extent = [c_lon_min, c_lon_max, c_lat_min, c_lat_max]

        try:
            radar = pyart.io.read(fn)
        except Exception as e:
            print(f"⚠️  Could not read {os.path.basename(fn)}: {e}")
            continue
        if field not in radar.fields:
            print(f"⚠️  {field} missing in {os.path.basename(fn)}, skipping")
            continue

        
        if field in radar.fields:
            field_data = radar.fields[field]['data']
            if isinstance(field_data, ma.MaskedArray):
                print(f"{field} frame {i}: min={field_data.min()}, max={field_data.max()}, masked={field_data.mask.sum()} / {field_data.size}")
            else:
                print(f"{field} frame {i}: min={np.min(field_data)}, max={np.max(field_data)}")

        fig = plt.figure(figsize=figsize, dpi=dpi)
        ax = plt.subplot(projection=ccrs.PlateCarree())
        ax.set_extent(extent, crs=ccrs.PlateCarree())

        # Add map features
        ax.add_feature(cfeature.COASTLINE.with_scale(map_detail))
        ax.add_feature(cfeature.BORDERS.with_scale(map_detail))
        ax.add_feature(cfeature.STATES.with_scale(map_detail))

        # Plot roads
        if road_gdf is not None:
            road_gdf.plot(ax=ax, color='dimgray', linewidth=0.4, transform=ccrs.PlateCarree(), zorder=3)

        # Plot water
        if water_gdf is not None:
            water_gdf.plot(ax=ax, color='dodgerblue', linewidth=0.6, alpha=0.6, transform=ccrs.PlateCarree(), zorder=2)

        if city_gdf is not None:
            # Marker size scales with zoom: larger markers for small domains
            city_size = 10 if domain_km <= 30 else 4
            city_gdf.plot(ax=ax, color='black', markersize=city_size, transform=ccrs.PlateCarree(), zorder=4)

            # Filter cities within current frame extent
            lon_min_cities, lon_max_cities, lat_min_cities, lat_max_cities = extent
            visible_cities = city_gdf.cx[lon_min_cities:lon_max_cities, lat_min_cities:lat_max_cities]

            # Always label cities within current extent
            for _, row in visible_cities.iterrows():
                name = row.get('name')
                if name:
                    ax.text(
                        row.geometry.x,
                        row.geometry.y,
                        name,
                        transform=ccrs.PlateCarree(),
                        fontsize=7 if domain_km <= 30 else 5,
                        ha='left',
                        va='bottom',
                        color='black',
                        zorder=5
                    )

        # Plot radar data
        disp = pyart.graph.RadarMapDisplay(radar)

        field_title_dict = {
            'reflectivity': 'Reflectivity',
            'velocity': 'Radial Velocity',
            'cross_correlation_ratio': 'Correlation Coefficient'
        }

        field_label_dict = {
            'reflectivity': 'Reflectivity (dBZ)',
            'velocity': 'Radial Velocity (m/s)',
            'cross_correlation_ratio': 'Correlation Coefficient'
        }

        title_text = field_title_dict.get(field)
        colorbar_label = field_label_dict.get(field)

        # Plot radar data but suppress Py-ART's default colorbar
        mesh = disp.plot_ppi_map(
            field, sweep,
            ax=ax,
            vmin=vmin, vmax=vmax, cmap=cmap,
            lat_lines=None, lon_lines=None,
            projection=ccrs.PlateCarree(),
            min_lat=c_lat_min, max_lat=c_lat_max,
            min_lon=c_lon_min, max_lon=c_lon_max,
            resolution=map_detail,
            colorbar_flag=False
        )

        # Add our custom colorbar
        norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
        mappable = cm.ScalarMappable(norm=norm, cmap=cmap)
        mappable.set_array([])  # Required to make the colorbar work

        cb = plt.colorbar(mappable, ax=ax, shrink=0.7, pad=0.02)
        cb.set_label(colorbar_label, fontsize=8)

        # Add gridlines
        add_gridlines(ax, extent)

        plt.title(f"{title_text} at {t:%Y-%m-%d %H:%M} UTC", fontsize=10)
        out_png = os.path.join(frames_dir, f"frame_{i:03d}.png")
        plt.savefig(out_png, bbox_inches=bbox_inches, pad_inches=0)
        plt.close(fig)

        frame_files.append(out_png)

    if not frame_files:
        print(f"⚠️  All files in the window were missing field `{field}`. No frames to stitch.")
        return

    imgs = [imageio.imread(fn) for fn in frame_files]
    gif_path = os.path.join(output_dir, f"{field}_{start_time:%H%M}_{end_time:%H%M}.gif")
    imageio.mimsave(gif_path, imgs, fps=4)
    print(f"✅  Saved {len(frame_files)} frames + GIF → {gif_path}")


In [ ]:
for field, cmap, vmin, vmax, folder, sweep in zip(['reflectivity', 'velocity', 'cross_correlation_ratio'], ['pyart_NWSRef', 'pyart_NWSVel', 'pyart_Carbone42'], [-20, -30, 0], [75, 30, 1], ['./radar_animations/tor/reflectivity', './radar_animations/tor/velocity', './radar_animations/tor/correlation'], [0, 1, 0]):
    plot_field_sequence(
        data_dir   = './radar_files',
        output_dir = folder,
        field      = field,
        start_time = datetime(2024,8,17,22,00),
        end_time   = datetime(2024,8,17,23,20),
        lat_min    = 44.5,
        lat_max    = 45,
        lon_min    = -122.5,
        lon_max    = -121.75,
        lon_rate   = 0,
        cmap       = cmap,
        vmin       = vmin,
        vmax       = vmax,
        features = True,
        figsize = (6, 6),
        sweep      = sweep,
        bbox_inches= 'tight'
    )
    

Size:  59.122932520638514 km
2024-08-17 22:03:00
reflectivity frame 0: min=-32.0, max=68.0, masked=14095223 / 16488000
2024-08-17 22:08:00
reflectivity frame 1: min=-32.0, max=68.0, masked=14069216 / 16488000


In [ ]:

for field, cmap, vmin, vmax, folder, sweep in zip(['reflectivity', 'velocity', 'cross_correlation_ratio'], ['pyart_NWSRef', 'pyart_NWSVel', 'pyart_Carbone42'], [-20, -30, 0], [75, 30, 1], ['./radar_animations/follow/reflectivity', './radar_animations/follow/velocity', './radar_animations/follow/correlation'], [0, 1, 0]):
    plot_field_sequence(
        data_dir   = './radar_files',
        output_dir = folder,
        field      = field,
        start_time = datetime(2024,8,17,21,00),
        end_time   = datetime(2024,8,18,3,00),
        lat_min    = 43.5,
        lat_max    = 44.5,
        lon_min    = -123,
        lon_max    = -121.5,
        lat_rate   = .0435,
        lon_rate   = .00725,
        cmap       = cmap,
        vmin       = vmin,
        vmax       = vmax,
        features   = True,
        sweep      = sweep
    )
    

Size:  448.9950000000002 km


c:\Users\miles\miniconda3\envs\mcs2\Lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


c:\Users\miles\miniconda3\envs\mcs2\Lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


c:\Users\miles\miniconda3\envs\mcs2\Lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


2024-08-17 21:01:00
reflectivity frame 0: min=-30.5, max=53.0, masked=12471222 / 14509440
2024-08-17 21:05:00
reflectivity frame 1: min=-31.5, max=60.5, masked=12453218 / 14509440
2024-08-17 21:10:00
reflectivity frame 2: min=-32.0, max=61.5, masked=13083997 / 15168960
2024-08-17 21:14:00
reflectivity frame 3: min=-30.5, max=59.0, masked=12434311 / 14509440
2024-08-17 21:18:00
reflectivity frame 4: min=-30.0, max=56.0, masked=12435813 / 14509440
2024-08-17 21:22:00
reflectivity frame 5: min=-32.0, max=55.0, masked=12427734 / 14509440
2024-08-17 21:26:00
reflectivity frame 6: min=-28.0, max=55.5, masked=12399847 / 14509440
2024-08-17 21:30:00
reflectivity frame 7: min=-29.5, max=58.5, masked=13647721 / 15828480
2024-08-17 21:35:00
reflectivity frame 8: min=-30.5, max=59.0, masked=12993318 / 15168960
2024-08-17 21:39:00
reflectivity frame 9: min=-32.0, max=59.5, masked=14249762 / 16488000
2024-08-17 21:44:00
reflectivity frame 10: min=-29.5, max=62.0, masked=12926962 / 15168960
2024-08-1

...

c:\Users\miles\miniconda3\envs\mcs2\Lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


c:\Users\miles\miniconda3\envs\mcs2\Lib\site-packages\osmnx\_overpass.py:267: UserWarning: This area is 28 times your configured Overpass max query area size. It will automatically be divided up into multiple sub-queries accordingly. This may take a long time.
  multi_poly_proj = utils_geo._consolidate_subdivide_geometry(poly_proj)


2024-08-17 21:01:00
cross_correlation_ratio frame 0: min=0.2083333283662796, max=1.0516666173934937, masked=13355184 / 14509440
2024-08-17 21:05:00
cross_correlation_ratio frame 1: min=0.2083333283662796, max=1.0516666173934937, masked=13341494 / 14509440
2024-08-17 21:10:00
cross_correlation_ratio frame 2: min=0.2083333283662796, max=1.0516666173934937, masked=13973904 / 15168960
2024-08-17 21:14:00
cross_correlation_ratio frame 3: min=0.2083333283662796, max=1.0516666173934937, masked=13332603 / 14509440
2024-08-17 21:18:00
cross_correlation_ratio frame 4: min=0.2083333283662796, max=1.0516666173934937, masked=13331138 / 14509440
2024-08-17 21:22:00
cross_correlation_ratio frame 5: min=0.2083333283662796, max=1.0516666173934937, masked=13322621 / 14509440
2024-08-17 21:26:00
cross_correlation_ratio frame 6: min=0.2083333283662796, max=1.0516666173934937, masked=13308838 / 14509440
2024-08-17 21:30:00
cross_correlation_ratio frame 7: min=0.2083333283662796, max=1.0516666173934937, mas

C:\Users\miles\AppData\Local\Temp\ipykernel_11768\1570463269.py:300: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = [imageio.imread(fn) for fn in frame_files]


✅  Saved 71 frames + GIF → ./radar_animations/follow/correlation\cross_correlation_ratio_2100_0300.gif
